In [1]:
import pandas as pd

DB_URL = 'onad.cbjjamtlar2t.ap-northeast-2.rds.amazonaws.com'
DB_USER = 'onad'
DB_PASSWORD = 'rkdghktn12'
DB_DATABASE = 'onad'
DB_CHARSET = 'utf8mb4'
DB_LOGFLAG  = 'False'
DB_PORT = 3306

db_url = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    DB_USER, DB_PASSWORD,
    DB_URL, DB_PORT,
    DB_DATABASE, DB_CHARSET
)

def data_from_sql_twitchchat(streamer, target_date):
    query = """
        select * from twitch_chat where streamer_name = "%s" and broad_date ="%s"
    """% (streamer, target_date)

    return pd.read_sql_query(query, db_url)

### 1 채팅데이터 로드

In [217]:
streamer = "rhdgurwns"
target_date = "2018-12-09"
df = data_from_sql_twitchchat(streamer, target_date)

In [218]:
chat_df = df.copy()
chat_df.head(2)

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
0,5804896,rhdgurwns,2018-12-09,<명부러전 (worldbest29)>,08:02:31,공혁준 방송켜라
1,5804897,rhdgurwns,2018-12-09,<명부러전 (worldbest29)>,08:02:39,방송켜 뽀뽀 쪽


In [219]:
chat_df.shape

(57597, 6)

### 2 시간 인덱스로 변환

In [222]:
# "얍하" 를 기준으로 시작시간 설정
start_time = chat_df[chat_df.chat_contents == "혁하"]["chat_time"].values[5]
start_time = target_date + " " + start_time
start_time = pd.to_datetime(start_time)

In [227]:
chat_df[chat_df.chat_contents == "혁하"].head()

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
stream_time,,,,,,
00:00:04,5804983,rhdgurwns,2018-12-09,<dlcjsdnjs2>,13:25:54,혁하
00:00:26,5804998,rhdgurwns,2018-12-09,<desiremon>,13:26:16,혁하
00:00:31,5805000,rhdgurwns,2018-12-09,<아무이유 (noreason_s)>,13:26:21,혁하
00:01:02,5805029,rhdgurwns,2018-12-09,<ssn065>,13:26:52,혁하
00:01:04,5805031,rhdgurwns,2018-12-09,<슈퍼똘똘 (jins1124)>,13:26:54,혁하


In [228]:
chat_df['stream_time'] = chat_df['chat_time'].apply(lambda x : target_date + " " + x)
chat_df['stream_time'] = chat_df['stream_time'].apply(lambda x : pd.to_datetime(x) - start_time)
chat_df.set_index("stream_time", inplace=True)

In [229]:
chat_df = chat_df[chat_df.index > "00:00:00"]

In [230]:
chat_df.head()

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
stream_time,,,,,,
00:00:01,5804981,rhdgurwns,2018-12-09,<어썸한어썸 (rewrite777)>,13:25:51,오늘 길게한다고 했으니까 최소 저녁8시까지 해라
00:00:02,5804982,rhdgurwns,2018-12-09,<혁준이가슴만지고싶다 (qkpa471)>,13:25:52,종아리 걷어 ㅡㅡ
00:00:04,5804983,rhdgurwns,2018-12-09,<dlcjsdnjs2>,13:25:54,혁하
00:00:04,5804984,rhdgurwns,2018-12-09,<바비노 (lowe3100)>,13:25:54,종아리걷어
00:00:05,5804985,rhdgurwns,2018-12-09,<kk99188>,13:25:55,혀카


### 3 분석데이터셋으로 전처리

#### 3-1 분석 데이터셋 만들기

In [231]:
anal_df = chat_df.pivot_table(index=chat_df.index, values='chatterer', aggfunc=len).loc[:, ['chatterer']]

#### 3-2 변수 추가

In [232]:
word_list = ['ㅋ', '유하', '꿀잼', '?', 'ㄵ', '노잼', 'ㄴㅈ', '성태', '얍마이무']

for word in word_list:
    target_word = "cnt_" + word
    chat_df[target_word] =  chat_df.chat_contents.apply(lambda x : x.count(word))
    anal_df[target_word] = chat_df.pivot_table(index=chat_df.index, values=target_word, aggfunc=sum).loc[:, target_word]

anal_df['cnt_ㅋ'].unique()

array([  0,   8,  13,   7,   5,   3,  14,   6,   4,   2,  11,  15,  25,
        20,  29,  19,  36,  12,  10,   1,  18,  30,  27,  23,  21,   9,
        39,  17,  28,  22,  24,  31,  32,  16,  26,  38,  40,  33,  37,
        35,  58,  81,  43,  55,  44,  47,  52,  54,  41,  68, 131,  78,
        34,  88,  80,  53,  60,  45,  57,  46,  49,  50,  62,  42,  82,
       112,  64, 106,  48,  92, 264, 140, 113, 168,  86, 109,  72,  74,
       191,  65,  63,  66, 167, 125, 156,  61, 132,  95,  90,  56,  69,
        85, 119,  93, 124, 108, 102,  77,  73, 346, 208, 216,  59, 103,
       139, 115, 157,  99,  84, 104, 218, 114, 117,  79, 100,  51, 154,
       160, 174, 101, 195, 192, 198, 107, 110,  97,  71, 122, 158, 419,
       380,  91, 166,  87,  98,  75, 196,  83, 128,  89, 150,  70,  67,
       194, 189, 249, 161, 193, 277, 126, 143, 141,  94, 120, 123, 111,
       250, 130, 303, 179, 145, 153, 142, 136, 121, 134, 144, 201, 137,
        76, 185, 116, 186,  96, 151, 227, 138, 172, 149, 184, 26

In [246]:
#Index(['2018-12-09 17:33:35', '2018-12-09 18:33:48', '2018-12-09 18:32:40',
#       '2018-12-15 14:29:25', '2018-12-09 18:33:25', '2018-12-09 18:34:03',
#        '2018-12-09 18:34:00', '2018-12-09 18:33:56', '2018-12-09 16:56:45'],
#       dtype='object', name='time')
chat_df[chat_df.index == '18:33:48']

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents,cnt_ㅋ,cnt_유하,cnt_꿀잼,cnt_?,cnt_ㄵ,cnt_노잼,cnt_ㄴㅈ,cnt_성태,cnt_얍마이무
stream_time,,,,,,,,,,,,,,,


In [429]:

from matplotlib import pyplot as plt
import time

def making_data():
    
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline, make_pipeline
    from sklearn.preprocessing import MinMaxScaler

    from sklearn.svm import SVC
    from sklearn import model_selection, svm, metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import f1_score
    import os
    
    
    chat_data = os.listdir("./data")
    datas = []
    for data in chat_data:
        tmp = pd.read_csv('./data/{}'.format(data))
        datas.append(tmp)
    
    df = pd.concat(datas , axis=0, join='inner', ignore_index=True)
    df = df.set_index(keys=df.columns[0])
    
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:, -1], random_state=0)
    anal_data = X_test[:]
    anal_data["validation"] = y_test
    
    
    scaler = MinMaxScaler()
    X_train = scaler.fit(X_train).transform(X_train)
    X_test = scaler.fit(X_test).transform(X_test)
    
    
    
    classifier = [LogisticRegression(), DecisionTreeClassifier(max_depth=10),
                 RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
                 GaussianNB()]
    
    classifier_score = {}
    
    for classify in classifier:
        clf = classify
        clf.fit(X_train,  y_train)
        pre = clf.predict(X_test)
        anal_data["{}".format(str(classify).split("(")[0])] = pre
        
        ac_score = metrics.accuracy_score(y_test, pre)
        f1 = f1_score(y_test, pre)
        
        classifier_score["{}".format(str(classify).split("(")[0])] = [ac_score, f1,
                                                                     len(anal_data[anal_data["{}".format(str(classify).split("(")[0])] == 1]) / len(anal_data[anal_data["validation"] == 1]) * 100]
        clf = None
        
        
    
    return classifier_score, anal_data

score, result1 = making_data()

result = result1.copy()

import pandas as pd
result.index = pd.to_datetime(result.index)
result1.index = pd.to_datetime(result1.index)

result

result = result.sort_index()
result1 = result1.sort_index()

result['날짜'] = result.index
result1['날짜'] = result1.index

result['날짜'] = result['날짜'].apply(lambda x : str(x.date()))
result1['날짜'] = result1['날짜'].apply(lambda x : str(x.date()))

result = result[result['날짜'] == "2018-12-15"]
result1 = result1[result1['날짜'] == "2018-12-15"]

yes_data = result[result.GaussianNB == 1]
yes_data1 = result[result.validation == 1]

no_data = result[result.GaussianNB == 0]
no_data1 = result[result.validation == 0]

plt.figure(figsize=(16,4))
plt.plot(no_data.index, no_data['cnt_chat'])

plt.plot(yes_data.index, yes_data['cnt_chat'])

# plt.plot(no_data1.index, no_data1['cnt_chat'])

plt.plot(yes_data1.index, yes_data1['cnt_chat'])

plt.legend()
plt.show()

yes_data = result1[result1.validation == 1]
no_data = result1[result1.validation == 0]

plt.figure(figsize=(16,4))
plt.plot(no_data.index, no_data['cnt_chat'])
plt.plot(yes_data.index, yes_data.cnt_chat)
plt.legend()
plt.show()